In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from mpl_toolkits.basemap import Basemap, addcyclic
%matplotlib qt
pd.set_option('mode.chained_assignment',None)

In [4]:
#set path
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [3]:
#set coordinates
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
#set fileneames and load temp extreme data
# file_era_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10.nc'.format(lon_0,lon_1,lat_0,lat_1)
# file_era_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90.nc'.format(lon_0,lon_1,lat_0,lat_1)
# file_hw_pers = 'persistent_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# file_hw_short = 'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# file_cw_pers = 'persistent_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# file_cw_short = 'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# file_independent_hw_short = 'independent_short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# file_independent_cw_short = 'independent_short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
# #load anomaly data
# era_anom_p90 = xr.open_dataset(path+file_era_anom_p90).squeeze()
# era_anom_p10 = xr.open_dataset(path+file_era_anom_p10).squeeze()

Heatwaves

In [19]:
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    file_era_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
    file_era_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
    era_anom_p90 = xr.open_dataset(path+file_era_anom_p90).squeeze()
    era_anom_p10 = xr.open_dataset(path+file_era_anom_p10).squeeze()
    file_hw_pers = 'persistent_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_0-2,lat_1)
    file_hw_short = 'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_0-2,lat_1)
    file_independent_hw_short = 'independent_short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_0-2,lat_1)

    #Load heat wave dates as np array
    pers_hw = np.load(path + file_hw_pers)
    short_hw = np.load(path + file_hw_short)
    independent_short_hw = np.load(path+file_independent_hw_short)
    #set columns for pandas dataframe
    columns = ['start_date','end_date','length','mean_anomaly','max_anomaly']

    file_namez = [file_hw_pers,file_hw_short]
    #loop over short and persistent heat waves
    variablez = [pers_hw,short_hw]
    for j,variable in enumerate(variablez):
        #set index as 1,2,3...
        index = np.arange(len(variable))
        #create empty dataframe
        df = pd.DataFrame(index=index,columns=columns)
        #loop over all temperature extremes
        for i in range(len(variable)):
            #select specific dates where heat wave occurs
            ds = era_anom_p90.sel(time=slice(variable[i,0],variable[i,1]))
            #caluclate mean anomaly from 90th percentile
            ds_mean = ds.t.mean().values
            #caluclate max anomaly from 90th percentile
            ds_max = ds.t.max().values
            #calculate length of heat wave
            length = len(ds.t)
            #set start and end dates
            start = variable[i,0]-pd.Timedelta(days=0)
            end = variable[i,1]-pd.Timedelta(days=0)
            #fill in dataframe
            df['start_date'][i]=start
            df['end_date'][i]=end
            df['length'][i]=length
            df['mean_anomaly'][i]=ds_mean
            df['max_anomaly'][i]=ds_max
            #for short heat waves fill in whether event is independent
            if j==1:
                df['independent']=independent_short_hw
            #save to csv 
            df.to_csv(path+file_namez[j][:-4]+'.csv')
    


Coldwaves

In [6]:
file_cw_pers = 'persistent_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
file_cw_short = 'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)

file_independent_cw_short = 'independent_short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)

#Load coldwave data
pers_cw = np.load(path + file_cw_pers)
short_cw = np.load(path + file_cw_short)
independent_short_cw = np.load(path+file_independent_cw_short)

#set columns for pandas dataframe
columns = ['start_date','end_date','length','mean_anomaly','min_anomaly']

file_namez = [file_cw_pers,file_cw_short]
variablez = [pers_cw,short_cw]
for j,variable in enumerate(variablez):
    #set index as 1,2,3...
    index = np.arange(len(variable))
    #create empty pandas dataframe
    df = pd.DataFrame(index=index,columns=columns)
    for i in range(len(variable)):
        #select specific dates where cold wave occurs
        ds = era_anom_p10.sel(time=slice(variable[i,0],variable[i,1]))
        #caluclate mean anomaly from 10th percentile
        ds_mean = ds.t.mean().values
        #calculate max anomaly from 10th percentile
        ds_min = ds.t.min().values
        #calculate length of anomaaly
        length = len(ds.t)
        #list start and end dates of cold wave
        start = variable[i,0]-pd.Timedelta(days=0)
        end = variable[i,1]-pd.Timedelta(days=0)
        #fill in dataframe
        df['start_date'][i]=start
        df['end_date'][i]=end
        df['length'][i]=length
        df['mean_anomaly'][i]=ds_mean
        df['min_anomaly'][i]=ds_min
        #for short cold waves list whether event is independent
        if j==1:
            df.loc[:,'independent']=independent_short_cw
        #save as csv file
        df.to_csv(path+file_namez[j][:-4]+'.csv')